Note download data from https://drive.google.com/drive/folders/1EgDN57LDuvlZAwr5-eHWB5CTJ7K9HpDP

Credit to this repo: https://github.com/LukasMosser/geolink_dataset

## Data Disclaimer

All the data serving as an input to these notebooks was generously donated by GEOLINK  
and is CC-by-SA 4.0 

If you use their data please reference their dataset properly to give them credit for their contribution.

In [1]:
import lasio
import matplotlib.pyplot as plt
%matplotlib inline
import os
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn import preprocessing
from operator import itemgetter

In [38]:
data_locations = Path("data/raw/geolink_dataset/GEOLINK North sea wells with Lithology interpretation/GEOLINK_Lithology and wells NORTH SEA")
interim_locations = Path("data/interim/geolink_dataset/")

In [41]:
df_lithology = pd.read_excel(data_locations/'../Lithology code data.xlsx', header=1)[:-1]
df_lithology['Abbreviation'] = pd.to_numeric(df_lithology['Abbreviation'])
df_lithology.to_parquet(interim_locations/'geolink_norge_lithology.parquet', compression='gzip')
df_lithology

,Lithology,Color,Lithology Attribute,Abbreviation
0,Aeolian Sandstone,LightYellow,CrossBedded Sand,35
1,Anhydrite,Light Magenta,Anhydrite,22
2,Argillaceous Limestone,Dodger Blue,Chalk,12
3,Arkose,LightGoldenrod,Gravel,36
4,Basement,Salmon,Intrusive,23
5,Biogenic Ooze,DarkYellow,Sandy Shale,25
6,Calcareous Cement,Cyan,Sandy Limestone,16
7,Calcareous Debris Flow,Turquoise,Breccia,31
8,Calcareous Shale,DarkCyan,Calcareous Shale,14
9,Carnallite,Magenta,Halite,33


In [42]:
df_picks = pd.read_excel(data_locations/'../NPD stratigraphic picks north sea.xlsx', header=0)
df_picks.to_parquet(interim_locations/'geolink_norge_picks.parquet', compression='gzip')

We can now proceed to import these files as las files and get their dataframes and hopefully put them into a data format that is more suited for ML tasks.

In [150]:

well_dataframes = []
files = sorted(data_locations.glob('*.las'))
for f in tqdm(files):
    df = lasio.read(f).df()
    df['Well'] = f.stem
    well_dataframes.append(df)

/home/wassname/.pyenv/versions/jup3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [151]:
df_all = pd.concat(well_dataframes)

df_all['Well'] = df_all['Well'].astype('category')

# Name lithology
litho_dict = df_lithology.set_index('Abbreviation')['Lithology'].to_dict()
df_all['LITHOLOGY_GEOLINK'] = df_all['LITHOLOGY_GEOLINK'].replace(litho_dict).astype('category')

# unique index
df_all = df_all.reset_index()#.set_index(['Well', 'DEPT'])

df_all

,DEPT,LITHOLOGY_GEOLINK,CALI,DRHO,NPHI,RHOB,GR,DTC,DTE,RDEP,...,SGR,ROP,DTS,MUDWEIGHT,RHOM,THOR,URAN,RMIC,CGR,TGAS
0,493.493134,NaN,19.406000,-0.708,NaN,1.789,64.497482,167.582153,50815100.0,1.574993,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,493.645538,NaN,19.406000,-0.722,NaN,1.754,62.406261,167.425064,50761636.0,1.569011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,493.797943,NaN,19.406000,-0.620,NaN,1.778,62.629055,167.808395,50886056.0,1.578010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,493.950348,NaN,19.452999,-0.648,NaN,1.642,65.998596,169.244873,51314492.0,1.586024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,494.102722,NaN,19.452999,-0.685,NaN,1.563,64.997223,170.635086,51763464.0,1.603011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3147157,4583.783691,NaN,9.596000,NaN,-0.05749,NaN,0.011769,NaN,NaN,1525.433960,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3147158,4583.936035,NaN,9.600000,NaN,-0.08727,NaN,-0.004249,NaN,NaN,1522.711060,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3147159,4584.088379,NaN,9.603000,NaN,-0.09888,NaN,-0.031477,NaN,NaN,1519.989014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3147160,4584.240723,NaN,9.607000,NaN,-0.10097,NaN,-0.061948,NaN,NaN,1517.265991,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [157]:
df_all.to_parquet(interim_locations/'geolink_norge_well_logs_raw.parquet', compression='gzip')

In [183]:
# Clean. must have lithology. Remove logs which are present less than half the time
df_all_clean2 = df_all.dropna(subset=['LITHOLOGY_GEOLINK'])
print(df_all_clean2.isna().mean().sort_values())
df_all_clean1 = df_all_clean2.dropna(axis=1, thresh=0.9*len(df_all_clean))
print(f'kept {len(df_all_clean1.columns)/len(df_all_clean2.columns):%} cols')
df_all_clean = df_all_clean1.dropna(axis=0)
df_all_clean.to_parquet(interim_locations/'geolink_norge_well_logs_clean.parquet', compression='gzip')
print(f'kept {len(df_all_clean)/len(df_all_clean2):%} rows')
df_all_clean

DEPT                 0.000000
Well                 0.000000
LITHOLOGY_GEOLINK    0.000000
GR                   0.010837
RDEP                 0.022700
CALI                 0.050810
DTC                  0.053175
RMED                 0.062857
RHOB                 0.067850
DRHO                 0.142370
NPHI                 0.235756
SP                   0.300878
RSHA                 0.316865
BS                   0.554892
PEF                  0.662368
DTE                  0.662811
ROP                  0.817933
DCAL                 0.825841
DTS                  0.869548
RMIC                 0.900082
URAN                 0.922957
THOR                 0.922972
MUDWEIGHT            0.957331
SGR                  0.958548
RXO                  0.966190
DT                   0.970601
RHOM                 0.980990
CGR                  0.996408
TGAS                 0.998670
dtype: float64
kept 44.827586% cols
kept 45.335716% rows


,DEPT,LITHOLOGY_GEOLINK,CALI,DRHO,NPHI,RHOB,GR,DTC,RDEP,SP,RSHA,RMED,Well
11302,2215.917725,Shaly Silt,14.438001,0.010000,0.447725,2.363000,60.285748,134.253601,0.737006,50.570400,0.785088,0.785088,15_9-12
11303,2216.070068,Shaly Silt,14.633000,0.006000,0.432095,2.340000,63.250000,129.101868,0.741000,50.820000,0.840000,0.840000,15_9-12
11304,2216.222412,Shaly Silt,14.813001,0.006000,0.410166,2.314000,61.405998,122.476944,0.752000,50.820000,0.858000,0.858000,15_9-12
11305,2216.375000,Shaly Silt,14.383001,-0.001000,0.414143,2.293000,62.561596,116.908607,0.739962,51.070000,0.857046,0.857046,15_9-12
11306,2216.527344,Shaly Silt,14.202999,0.001000,0.439923,2.275000,61.691055,115.390953,0.715966,51.070602,0.886082,0.886082,15_9-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3134951,2723.585938,Anhydrite,9.157000,0.141856,0.000000,2.692753,15.846580,53.005497,1504.687866,59.057552,14.382000,1453.242920,7_3-1
3134952,2723.738525,Anhydrite,9.197000,0.148906,0.000000,2.695477,16.209375,53.386738,1514.930054,58.484932,14.387000,1471.283691,7_3-1
3134953,2723.890869,Anhydrite,9.236000,0.148817,0.000000,2.701625,16.638577,53.277893,1522.655029,58.483002,14.390000,1473.072998,7_3-1
3134954,2724.043213,Anhydrite,9.262000,0.150856,0.000000,2.702774,16.997316,52.928886,1530.355835,58.843422,14.390000,1473.072998,7_3-1


In [184]:
df_all_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 748470 entries, 11302 to 3134955
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   DEPT               748470 non-null  float64 
 1   LITHOLOGY_GEOLINK  748470 non-null  category
 2   CALI               748470 non-null  float64 
 3   DRHO               748470 non-null  float64 
 4   NPHI               748470 non-null  float64 
 5   RHOB               748470 non-null  float64 
 6   GR                 748470 non-null  float64 
 7   DTC                748470 non-null  float64 
 8   RDEP               748470 non-null  float64 
 9   SP                 748470 non-null  float64 
 10  RSHA               748470 non-null  float64 
 11  RMED               748470 non-null  float64 
 12  Well               748470 non-null  category
dtypes: category(2), float64(11)
memory usage: 70.7 MB


In [200]:
# Test load
df = pd.read_parquet(interim_locations/'geolink_norge_well_logs_clean.parquet').set_index(['Well', 'DEPT'])

LITHOLOGY_GEOLINK       CALI      DRHO      NPHI  \
Well    DEPT                                                           
15_9-12 2215.917725        Shaly Silt  14.438001  0.010000  0.447725   
        2216.070068        Shaly Silt  14.633000  0.006000  0.432095   
        2216.222412        Shaly Silt  14.813001  0.006000  0.410166   
        2216.375000        Shaly Silt  14.383001 -0.001000  0.414143   
        2216.527344        Shaly Silt  14.202999  0.001000  0.439923   
...                               ...        ...       ...       ...   
7_3-1   2723.585938         Anhydrite   9.157000  0.141856  0.000000   
        2723.738525         Anhydrite   9.197000  0.148906  0.000000   
        2723.890869         Anhydrite   9.236000  0.148817  0.000000   
        2724.043213         Anhydrite   9.262000  0.150856  0.000000   
        2724.195557         Anhydrite   9.268000  0.145638  0.000000   

                         RHOB         GR         DTC         RDEP         SP  \
Well    DEPT                                                                   
15_9-12 2215.917725  2.363000  60.285748  134.253601     0.737006  50.570400   
        2216.070068  2.340000  63.250000  129.101868     0.741000  50.820000   
        2216.222412  2.314000  61.405998  122.476944     0.752000  50.820000   
        2216.375000  2.293000  62.561596  116.908607     0.739962  51.070000   
        2216.527344  2.275000  61.691055  115.390953     0.715966  51.070602   
...                       ...        ...         ...          ...        ...   
7_3-1   2723.585938  2.692753  15.846580   53.005497  1504.687866  59.057552   
        2723.738525  2.695477  16.209375   53.386738  1514.930054  58.484932   
        2723.890869  2.701625  16.638577   53.277893  1522.655029  58.483002   
        2724.043213  2.702774  16.997316   52.928886  1530.355835  58.843422   
        2724.195557  2.690725  17.194046   52.892044  1538.068726  59.203846   

                          RSHA         RMED  
Well    DEPT                                 
15_9-12 2215.917725   0.785088     0.785088  
        2216.070068   0.840000     0.840000  
        2216.222412   0.858000     0.858000  
        2216.375000   0.857046     0.857046  
        2216.527344   0.886082     0.886082  
...                        ...          ...  
7_3-1   2723.585938  14.382000  1453.242920  
        2723.738525  14.387000  1471.283691  
        2723.890869  14.390000  1473.072998  
        2724.043213  14.390000  1473.072998  
        2724.195557  14.389000  1473.072998  

[748470 rows x 11 columns]

In [195]:
df_well_heads = pd.concat([
    pd.read_csv(data_locations/'../../wellbore_exploration_all.csv'),
    pd.read_csv(data_locations/'../../wellbore_development_all.csv'),
    pd.read_csv(data_locations/'../../wellbore_other_all.csv'),
])

In [196]:
df_well_heads

,wlbWellboreName,wlbWell,wlbDrillingOperator,wlbProductionLicence,wlbPurpose,wlbStatus,wlbContent,wlbWellType,wlbSubSea,wlbEntryDate,...,wlbNpdidSiteSurvey,wlbDateUpdated,wlbDateUpdatedMax,datesyncNPD,wlbEntryPreDrillDate,wlbCompPreDrillDate,wlbProductionFacility,wlbContentPlanned,prlNpdidProdLicenceTarget,fclNpdidFacilityProducing
0,1/2-1,1/2-1,Phillips Petroleum Norsk AS,143,WILDCAT,P&A,OIL,EXPLORATION,NO,20.03.1989,...,NaN,03.10.2019,10.03.2020,03.07.2020,NaN,NaN,NaN,NaN,NaN,NaN
1,1/2-2,1/2-2,Paladin Resources Norge AS,143 CS,WILDCAT,P&A,OIL SHOWS,EXPLORATION,NO,14.12.2005,...,NaN,03.10.2019,03.10.2019,03.07.2020,NaN,NaN,NaN,NaN,NaN,NaN
2,1/3-1,1/3-1,A/S Norske Shell,011,WILDCAT,P&A,GAS,EXPLORATION,NO,06.07.1968,...,NaN,03.10.2019,10.03.2020,03.07.2020,NaN,NaN,NaN,NaN,NaN,NaN
3,1/3-2,1/3-2,A/S Norske Shell,011,WILDCAT,P&A,DRY,EXPLORATION,NO,14.05.1969,...,NaN,03.10.2019,10.03.2020,03.07.2020,NaN,NaN,NaN,NaN,NaN,NaN
4,1/3-3,1/3-3,Elf Petroleum Norge AS,065,WILDCAT,P&A,OIL,EXPLORATION,NO,22.08.1982,...,NaN,06.03.2020,10.03.2020,03.07.2020,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,8006/8-U-4,8006/8-U-4,OCEAN DRILLING PROGRAM TEXAS A & M UNIVERSITY,NaN,SCIENTIFIC,NaN,NaN,OTHER,NaN,29.08.1993,...,NaN,03.10.2019,18.10.2014,03.07.2020,NaN,NaN,NaN,NaN,NaN,NaN
1115,8008/7-U-1,8008/7-U-1,OCEAN DRILLING PROGRAM TEXAS A & M UNIVERSITY,NaN,SCIENTIFIC,NaN,NaN,OTHER,NaN,22.08.1993,...,NaN,03.10.2019,18.10.2014,03.07.2020,NaN,NaN,NaN,NaN,NaN,NaN
1116,8008/7-U-2,8008/7-U-2,OCEAN DRILLING PROGRAM TEXAS A & M UNIVERSITY,NaN,SCIENTIFIC,NaN,NaN,OTHER,NaN,22.08.1993,...,NaN,03.10.2019,18.10.2014,03.07.2020,NaN,NaN,NaN,NaN,NaN,NaN
1117,8008/7-U-3,8008/7-U-3,OCEAN DRILLING PROGRAM TEXAS A & M UNIVERSITY,NaN,SCIENTIFIC,NaN,NaN,OTHER,NaN,22.08.1993,...,NaN,03.10.2019,18.10.2014,03.07.2020,NaN,NaN,NaN,NaN,NaN,NaN


In [198]:
df_all['Well']

0          15_9-12
1          15_9-12
2          15_9-12
3          15_9-12
4          15_9-12
            ...   
3147157      7_3-1
3147158      7_3-1
3147159      7_3-1
3147160      7_3-1
3147161      7_3-1
Name: Well, Length: 3147162, dtype: category
Categories (223, object): [15_9-12, 15_9-13, 15_9-14, 15_9-15, ..., 6_3-2, 7_1-1, 7_1-2 S, 7_3-1]